In [1]:
import os
import pickle
import yaml
import functools

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

import ml_collections
from clu import checkpoint


from molnet import utils, train_state
from molnet.data import input_pipeline
from molnet.models import create_model
from configs import root_dirs
from analyses import make_predictions

from typing import Tuple

2024-11-08 13:21:39.740680: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
workdir = "/u/79/kurkil1/unix/work/molnet/local_runs/local"

In [3]:
def load_from_workdir(
    workdir: str,
    return_attention: bool
):
    # Load the model config
    with open(os.path.join(workdir, "config.yaml"), "rt") as f:
        config = yaml.unsafe_load(f)
    config = ml_collections.ConfigDict(config)
    config.root_dir = root_dirs.get_root_dir()
    config.model.return_attention_maps = return_attention

    # Create the model
    model = create_model(config.model)

    checkpoint_dir = os.path.join(workdir, "checkpoints")
    ckpt = checkpoint.Checkpoint(checkpoint_dir)

    apply_fn = model.apply
    tx = utils.create_optimizer(config)
    restored_state = ckpt.restore(state=None)['state']

    # Load the model state
    state = train_state.EvaluationState.create(
        apply_fn=apply_fn,
        params=restored_state['params'],
        batch_stats=restored_state['batch_stats'],
        tx=tx,
    )
    state = jax.tree_util.tree_map(jnp.asarray, state)

    return state, config

In [4]:
state, config = load_from_workdir(
    workdir=workdir,
    return_attention=True
)

2024-11-08 13:21:44.016647: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
rng = jax.random.PRNGKey(0)
datarng, rng = jax.random.split(rng)
ds = input_pipeline.get_datasets(
    rng,
    config
)['val']

batch = next(ds)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 16254440238521100737
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 9035647960369128274
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing t

In [6]:
@jax.jit
def predict(
    state,
    batch,
):
    inputs, targets = batch['images'], batch['atom_map']
    preds, attention = state.apply_fn(
        {'params': state.params, 'batch_stats': state.batch_stats},
        inputs,
        training=False,
    )
    preds_z = preds.shape[-2]
    target = targets[..., -preds_z:, :]
    loss_by_image = jnp.mean(
        (preds - target) ** 2,
        axis=(1, 2, 3, 4),
    )
    return inputs, target, preds, attention, loss_by_image


In [7]:
inputs, targets, preds, attention, loss_by_image = predict(
    state,
    batch,
)

In [8]:
print(f"inputs: {inputs.shape}")
print(f"targets: {targets.shape}")
print(f"preds: {preds.shape}")
print(f"loss_by_image: {loss_by_image.shape}")
for att in attention:
    print(f"attention: {att.shape}")

inputs: (8, 128, 128, 10, 1)
targets: (8, 128, 128, 10, 5)
preds: (8, 128, 128, 10, 5)
loss_by_image: (8,)
attention: (8, 8, 8, 10, 1)
attention: (8, 16, 16, 10, 1)
attention: (8, 32, 32, 10, 1)
attention: (8, 64, 64, 10, 1)
attention: (8, 128, 128, 10, 1)


In [10]:
make_predictions.make_predictions(
    workdir=workdir,
    outputdir=os.path.join(workdir, "analysis"),
    num_batches=1
)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 16254440238521100737
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 9035647960369128274
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing t